In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../../turksat-classifier/v1/dataset.csv").sample(frac=1)

In [ ]:
df

In [ ]:
# Load the library with the CountVectorizer method
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
# Helper function
def plot_10_most_common_words(count_data, count_vectorizer):
    import matplotlib.pyplot as plt
    words = count_vectorizer.get_feature_names()
    total_counts = np.zeros(len(words))
    for t in count_data:
        total_counts+=t.toarray()[0]
    
    count_dict = (zip(words, total_counts))
    count_dict = sorted(count_dict, key=lambda x:x[1], reverse=True)[0:10]
    words = [w[0] for w in count_dict]
    counts = [w[1] for w in count_dict]
    x_pos = np.arange(len(words)) 
    
    plt.figure(2, figsize=(15, 15/1.6180))
    plt.subplot(title='10 most common words')
    sns.set_context("notebook", font_scale=1.25, rc={"lines.linewidth": 2.5})
    sns.barplot(x_pos, counts, palette='husl')
    plt.xticks(x_pos, words, rotation=90) 
    plt.xlabel('words')
    plt.ylabel('counts')
    plt.show()
# Initialise the count vectorizer with the English stop words
count_vectorizer = CountVectorizer(stop_words='english')
# Fit and transform the processed titles
count_data = count_vectorizer.fit_transform(df['text'])
# Visualise the 10 most common words
plot_10_most_common_words(count_data, count_vectorizer)


In [ ]:
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
# Load the LDA model from sk-learn
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer

# Helper function
def get_topic_info(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    topic_info = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_info[topic_idx] = [words[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        
# Tweak the two parameters below
number_topics = 42
number_words = 10
# Create and fit the LDA model
lda = LDA(n_components=number_topics, n_jobs=-1)
m = lda.fit_transform(count_data)
# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, number_words)


In [ ]:
i = 22
print(np.argmax(m[i]))
count_vectorizer.inverse_transform(count_data[i])

In [ ]:
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer

def get_topic_info(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    topic_info = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_info[topic_idx] = [words[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        
    return topic_info
def topic_modeller(column, num_topics=10, num_words=10):
    count_vectorizer = CountVectorizer()
    count_data = count_vectorizer.fit_transform(column)
    lda = LDA(n_components=num_topics, n_jobs=-1)
    m = lda.fit_transform(count_data)
    topic_info = get_topic_info(lda, count_vectorizer, num_words)
    new_column = []
    for item in m:
        new_column.append(np.argmax(item))
    return new_column, topic_info


In [ ]:
topic_modeller(df["text"][:50])